In [ ]:
# Esse codigo é um exemplo de como criar um ambiente virtual e instalar dependências em Python.
#pip install -r requirements.txt

In [ ]:
# PRECISA INSTALAR O PYTHON-TK PARA RODAR ESSE CÓDIGO
# brew install python-tk

In [8]:
# Bibliotecas necessárias
import pandas as pd
import json
import cvxpy as cp




In [9]:
df = pd.read_excel('Data.xlsx')

In [19]:
# Caminho do arquivo Excel
excel_path = 'Data.xlsx'

# Carregar a planilha (assumindo que os dados estão na primeira aba)
df = pd.read_excel(excel_path)

# Remover espaços extras dos nomes das colunas (isso resolve problemas de leitura)
df.columns = df.columns.str.strip()

# Criar uma coluna identificadora única para cada linha (DMU)
df['DMU'] = df['Banco'] + '_' + df.index.astype(str)

# Selecionar colunas no formato exigido pelo pyDEA
df_dea = df[[
    'DMU',
    'Despesa de Intermediação Financeira',
    'Despesa Administrativa e Operacionais',
    'Receita de Intermediação Financeira',
    'Receita com prestação de serviços',
    'Outras receitas operacionais'
]]



In [21]:
# Salvar em CSV
df_dea.to_csv('dados_dea.csv', index=False)

In [ ]:
# Definir configurações do modelo DEA
config = {
    "input_categories": [
        "Despesa de Intermediação Financeira",
        "Despesa Administrativa e Operacionais"
    ],
    "output_categories": [
        "Receita de Intermediação Financeira",
        "Receita com prestação de serviços",
        "Outras receitas operacionais"
    ],
    "orientation": "input",          # orientação a input (minimizar despesas)
    "returns_to_scale": "CRS"        # Retornos constantes de escala
}

# Salvar em JSON
with open('config_dea.json', 'w') as f:
    json.dump(config, f, indent=4)